In [1]:
from autoencoder import Autoencoder
from parser import data_converter, print_letter
import numpy as np
import matplotlib.pyplot as plt

In [2]:
alphabet = np.array(data_converter("resources"))
# for char in alphabet:
#     print_letter(char)
# print(len(alphabet))
flattened_input = np.array(list(map(lambda char: np.array(char).flatten(), alphabet)))

In [3]:
epochs = []
errors =[]

def callback_fun(e, error, w, o):
    epochs.append(e)
    errors.append(error)

In [4]:

activation_function = (lambda x: 1/(1+np.exp(-2*x)))
activation_derivative = (lambda x: 2*(1/(1+np.exp(-2*x)))*(1-(1/(1+np.exp(-2*x)))) )
ae = Autoencoder(35, [], latent_dim=2, activation_function=activation_function,activation_function_derivative=activation_derivative, update_learn_rate=None, learning_rate=0.001, update_frequency=len(flattened_input),momentum=False, use_adam=True)


output, err_min = ae.train(flattened_input, expected_output=flattened_input, epoch_limit=1500, callback=callback_fun)

print(err_min)

1
dot product:  [[1.19006462 1.27012853 1.20778451]
 [1.27012853 1.3555789  1.28904056]
 [1.20778451 1.28904056 1.22576824]]


/Users/ana.cruz/Documents/ITBA/SIA/SIA-tps/TP5/multilayer_perceptron.py:123: RuntimeWarning: divide by zero encountered in true_divide
  m_t_hat = m_t/(1-beta_1**t)
/Users/ana.cruz/Documents/ITBA/SIA/SIA-tps/TP5/multilayer_perceptron.py:124: RuntimeWarning: divide by zero encountered in true_divide
  v_t_hat = v_t/(1-beta_2**t)


TypeError: only length-1 arrays can be converted to Python scalars

In [ ]:
plt.plot(epochs, errors)

In [ ]:
i = 10
print_letter(alphabet[i])
reconstruct, reconstruction_error = ae.reconstruct(flattened_input[i])

print_letter(reconstruct.reshape(alphabet[i].shape))
print(reconstruction_error)

latent_output = ae.encode(flattened_input[i])
print(latent_output)
print_letter(ae.decode(latent_output).reshape(alphabet[i].shape))

ae.graph_latent_space(flattened_input)

